# Simple Model

- We don't have returns after 2022 Jan. 
- We have data until 2022 Sept.

- For model training:
    - Testing period --> 2021 June - 2022 Jan
    - Train period: 2017 June - 2021 June (3 years)
    One stock per line. Aggregate all data into one row

- For Inference:
    - Infer for periods between 2019 Sept. - 2022 Sept.
    - This will give us the returns for 2022 Sept. until 2023 March. 
    - Choose the top 15 stocks



In [1]:
cd ..

/Users/guillaumedherouville/Library/CloudStorage/OneDrive-HECParis/HEC/quinten-stock-prediction


In [2]:
import pandas as pd
import numpy as np
import src.data.preprocessing as pr
import datetime as dt
from references.config import DATES
from xgboost import XGBRegressor

In [3]:
df = pr.get_data()

/Users/guillaumedherouville/Library/CloudStorage/OneDrive-HECParis/HEC/quinten-stock-prediction/src/data/preprocessing.py:7: DtypeWarning: Columns (8,17,26,33,34,66,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,252,253,255,257,259,261,263,265,267,269,270,271,272,273,274,275,276,277,278,279,280,284,285,286,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";", parse_dates=date_cols).iloc[:, 1:]
/Users/guillaumedherouville/Library/CloudStorage/OneDrive-HECParis/HEC/quinten-stock-prediction/src/data/preprocessing.py:47: FutureWarning: The default fill_method='pad' in Series.pct_chan

In [4]:
df[df.year==2022].groupby("quarter").apply(lambda x: len(x.index.get_level_values(1).unique()))

quarter
1    1407
3    1407
6    1407
9    1407
dtype: int64

In [5]:
df[df.year==2022].close.dropna()

date        symbol  
2022-01-31  0O9.F        11.952000
            0QF.F       148.268000
            217A.F        7.990000
            22UA.F      156.159998
            2CRSI.PA      4.443000
                           ...    
            XIL.PA       38.820000
            XIOR.BR      48.390000
            YATRA.AS      3.300000
            YK6B.F       15.350000
            ZEL.NZ        3.598000
Name: close, Length: 1387, dtype: float64

# Model: XGBoost 

__Only keep stocks that are trading in Euros
currency_by_stock = df["reportedCurrency"].groupby("symbol").apply(lambda x:x.dropna()[0] if len(x.dropna()) > 0 else np.nan)
euro_stocks = currency_by_stock[currency_by_stock=="EUR"].index
idx = pd.IndexSlice[:, euro_stocks]
df = df.loc[idx, :]

In [6]:
X_train = df.loc[DATES["train_start"]: DATES["train_end"]]
X_train = X_train.unstack(level=0).drop("year", axis=1)
X_train

quarter                                                         \
date     2015-06-30 2015-09-30 2016-01-31 2016-03-31 2016-06-30 2016-09-30   
symbol                                                                       
0O9.F             6          9          1          3          6          9   
0QF.F             6          9          1          3          6          9   
217A.F            6          9          1          3          6          9   
22UA.F            6          9          1          3          6          9   
2CRSI.PA          6          9          1          3          6          9   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR           6          9          1          3          6          9   
YATRA.AS          6          9          1          3          6          9   
YK6B.F            6          9          1          3          6          9   
ZEL.NZ            6          9          1          3          6          9   
ZEN.BR            6          9          1          3          6          9   

                                                      ...     return  \
date     2017-01-31 2017-03-31 2017-06-30 2017-09-30  ... 2016-03-31   
symbol                                                ...              
0O9.F             1          3          6          9  ...        NaN   
0QF.F             1          3          6          9  ...        NaN   
217A.F            1          3          6          9  ...        NaN   
22UA.F            1          3          6          9  ...        NaN   
2CRSI.PA          1          3          6          9  ...        NaN   
...             ...        ...        ...        ...  ...        ...   
XIOR.BR           1          3          6          9  ...        NaN   
YATRA.AS          1          3          6          9  ...        NaN   
YK6B.F            1          3          6          9  ...        NaN   
ZEL.NZ            1          3          6          9  ...        NaN   
ZEN.BR            1          3          6          9  ...        NaN   

                                                                            \
date     2016-06-30 2016-09-30 2017-01-31 2017-03-31 2017-06-30 2017-09-30   
symbol                                                                       
0O9.F           NaN        NaN        NaN        NaN        NaN        NaN   
0QF.F           NaN        NaN        NaN        NaN        NaN        NaN   
217A.F          NaN        NaN        NaN        NaN  -0.100321   0.119183   
22UA.F          NaN        NaN        NaN        NaN        NaN        NaN   
2CRSI.PA        NaN        NaN        NaN        NaN        NaN        NaN   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR         NaN        NaN        NaN        NaN   0.055538   0.094661   
YATRA.AS        NaN        NaN        NaN        NaN   0.067366  -0.240817   
YK6B.F          NaN        NaN        NaN        NaN        NaN        NaN   
ZEL.NZ          NaN        NaN        NaN        NaN   0.117664  -0.046393   
ZEN.BR          NaN        NaN        NaN        NaN   0.024316   0.045107   

                                           
date     2018-01-31 2018-03-31 2018-06-30  
symbol                                     
0O9.F           NaN        NaN        NaN  
0QF.F           NaN        NaN        NaN  
217A.F     0.401490  -0.209786   0.319929  
22UA.F          NaN        NaN        NaN  
2CRSI.PA        NaN        NaN        NaN  
...             ...        ...        ...  
XIOR.BR   -0.050507  -0.031325   0.066608  
YATRA.AS  -0.131175  -0.014706  -0.094129  
YK6B.F          NaN        NaN        NaN  
ZEL.NZ     0.009088  -0.080464   0.083471  
ZEN.BR    -0.038838   0.009618  -0.009180  

[1407 rows x 3705 columns]

In [7]:
# y_train = 
test_per_rets = df.loc[DATES["val_start"]: DATES["val_end"], "return"]
test_per_rets

date        symbol  
2018-06-30  0O9.F            NaN
            0QF.F            NaN
            217A.F      0.319929
            22UA.F           NaN
            2CRSI.PA         NaN
                          ...   
2019-01-31  XIOR.BR    -0.003489
            YATRA.AS    0.041420
            YK6B.F           NaN
            ZEL.NZ     -0.166667
            ZEN.BR     -0.018987
Name: return, Length: 4221, dtype: float64

In [8]:
# drop stocks with missing returns for this period
to_drop = test_per_rets[test_per_rets.isna()].index.get_level_values("symbol").unique()

# drop stocks that traded under 1$ in the period
under_1 = X_train['close'].groupby(level="symbol").apply(lambda x: (x<1).any(axis=1)).droplevel(1)
to_drop = to_drop.union(under_1[under_1].index).unique()

X_train = X_train.drop(to_drop)
y_train = test_per_rets.drop(to_drop, level=1)

In [9]:
print(np.setdiff1d(X_train.index, y_train.index.get_level_values(1)))
print(np.setdiff1d(y_train.index.get_level_values(1), X_train.index))

[]
[]


In [10]:
y_train = y_train.groupby(level=1, group_keys=False).apply(lambda x: x.cumprod()[-1])

assert all(X_train.index == y_train.index)

/var/folders/jm/2zytm77j0ms47q8tymwmv8800000gn/T/ipykernel_86187/2731682971.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train = y_train.groupby(level=1, group_keys=False).apply(lambda x: x.cumprod()[-1])


In [11]:
# Last checks and drops on columns
X_train = X_train.select_dtypes(exclude=["object", "datetime"])

In [12]:
X_train.dtypes.value_counts()


float64    3627
int64        26
Name: count, dtype: int64

In [13]:
X_train

quarter                                                         \
date     2015-06-30 2015-09-30 2016-01-31 2016-03-31 2016-06-30 2016-09-30   
symbol                                                                       
217A.F            6          9          1          3          6          9   
3E2.F             6          9          1          3          6          9   
A17U.SI           6          9          1          3          6          9   
A3M.MC            6          9          1          3          6          9   
A5G.IR            6          9          1          3          6          9   
...             ...        ...        ...        ...        ...        ...   
XIL.PA            6          9          1          3          6          9   
XIOR.BR           6          9          1          3          6          9   
YATRA.AS          6          9          1          3          6          9   
ZEL.NZ            6          9          1          3          6          9   
ZEN.BR            6          9          1          3          6          9   

                                                      ...     return  \
date     2017-01-31 2017-03-31 2017-06-30 2017-09-30  ... 2016-03-31   
symbol                                                ...              
217A.F            1          3          6          9  ...        NaN   
3E2.F             1          3          6          9  ...        NaN   
A17U.SI           1          3          6          9  ...        NaN   
A3M.MC            1          3          6          9  ...        NaN   
A5G.IR            1          3          6          9  ...        NaN   
...             ...        ...        ...        ...  ...        ...   
XIL.PA            1          3          6          9  ...        NaN   
XIOR.BR           1          3          6          9  ...        NaN   
YATRA.AS          1          3          6          9  ...        NaN   
ZEL.NZ            1          3          6          9  ...        NaN   
ZEN.BR            1          3          6          9  ...        NaN   

                                                                            \
date     2016-06-30 2016-09-30 2017-01-31 2017-03-31 2017-06-30 2017-09-30   
symbol                                                                       
217A.F          NaN        NaN        NaN        NaN  -0.100321   0.119183   
3E2.F           NaN        NaN        NaN        NaN   0.401353   0.089776   
A17U.SI         NaN        NaN        NaN        NaN   0.044806   0.013283   
A3M.MC          NaN        NaN        NaN        NaN  -0.111895  -0.143632   
A5G.IR          NaN        NaN        NaN        NaN  -0.031933   0.016942   
...             ...        ...        ...        ...        ...        ...   
XIL.PA          NaN        NaN        NaN        NaN   0.775747  -0.054249   
XIOR.BR         NaN        NaN        NaN        NaN   0.055538   0.094661   
YATRA.AS        NaN        NaN        NaN        NaN   0.067366  -0.240817   
ZEL.NZ          NaN        NaN        NaN        NaN   0.117664  -0.046393   
ZEN.BR          NaN        NaN        NaN        NaN   0.024316   0.045107   

                                           
date     2018-01-31 2018-03-31 2018-06-30  
symbol                                     
217A.F     0.401490  -0.209786   0.319929  
3E2.F      0.066852   0.459954   0.608037  
A17U.SI    0.031461  -0.049564   0.000955  
A3M.MC    -0.043685  -0.100444   0.005887  
A5G.IR     0.122441  -0.149589  -0.025045  
...             ...        ...        ...  
XIL.PA     0.845173  -0.009583   0.201489  
XIOR.BR   -0.050507  -0.031325   0.066608  
YATRA.AS  -0.131175  -0.014706  -0.094129  
ZEL.NZ     0.009088  -0.080464   0.083471  
ZEN.BR    -0.038838   0.009618  -0.009180  

[1104 rows x 3653 columns]

In [14]:
model = XGBRegressor()
model.fit(X_train.to_numpy(), y_train)

/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
#model.save_model()

In [16]:
model.score(X_train, y_train)

/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtyp

0.999107403954166

# Predicting Returns

In [17]:
X_test = df.loc[DATES["infer_start"]: DATES["infer_end"]]
X_test = X_test.unstack(level=0).drop("year", axis=1)
X_test

quarter                                                         \
date     2019-01-31 2019-03-31 2019-06-30 2019-09-30 2020-01-31 2020-03-31   
symbol                                                                       
0O9.F             1          3          6          9          1          3   
0QF.F             1          3          6          9          1          3   
217A.F            1          3          6          9          1          3   
22UA.F            1          3          6          9          1          3   
2CRSI.PA          1          3          6          9          1          3   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR           1          3          6          9          1          3   
YATRA.AS          1          3          6          9          1          3   
YK6B.F            1          3          6          9          1          3   
ZEL.NZ            1          3          6          9          1          3   
ZEN.BR            1          3          6          9          1          3   

                                                      ...     return  \
date     2020-06-30 2020-09-30 2021-01-31 2021-03-31  ... 2019-09-30   
symbol                                                ...              
0O9.F             6          9          1          3  ...   0.420561   
0QF.F             6          9          1          3  ...   0.143268   
217A.F            6          9          1          3  ...   0.034743   
22UA.F            6          9          1          3  ...        NaN   
2CRSI.PA          6          9          1          3  ...  -0.475438   
...             ...        ...        ...        ...  ...        ...   
XIOR.BR           6          9          1          3  ...   0.063473   
YATRA.AS          6          9          1          3  ...  -0.051896   
YK6B.F            6          9          1          3  ...        NaN   
ZEL.NZ            6          9          1          3  ...  -0.118660   
ZEN.BR            6          9          1          3  ...   0.001267   

                                                                            \
date     2020-01-31 2020-03-31 2020-06-30 2020-09-30 2021-01-31 2021-03-31   
symbol                                                                       
0O9.F      0.038158  -0.205070   1.377232   0.080483   1.379888  -0.230569   
0QF.F      0.432857   0.421137   0.880384   0.121367   1.177801  -0.179831   
217A.F     0.242336   0.471210   0.773163  -0.127928   0.657025  -0.150561   
22UA.F          NaN   0.792782   0.159442  -0.006257   0.518609  -0.002897   
2CRSI.PA  -0.098941  -0.498644   0.702118  -0.161769   0.490524   0.077559   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR    0.135666  -0.270473   0.230433   0.156309  -0.074919  -0.106025   
YATRA.AS   0.052632   0.030000  -0.076699  -0.100946   0.111111  -0.025000   
YK6B.F          NaN        NaN        NaN        NaN        NaN   0.099440   
ZEL.NZ    -0.189287  -0.350446  -0.063918   0.005140   0.105917  -0.058785   
ZEN.BR     0.228354  -0.094188   0.116041   0.033537   0.134234  -0.013043   

                                           
date     2021-06-30 2021-09-30 2022-01-31  
symbol                                     
0O9.F     -0.085763  -0.259548  -0.401502  
0QF.F      0.822632   0.673667  -0.547315  
217A.F    -0.313028  -0.225427  -0.448966  
22UA.F     1.111287   0.264526  -0.350037  
2CRSI.PA  -0.172468  -0.034221   0.093258  
...             ...        ...        ...  
XIOR.BR    0.114661  -0.026894  -0.023805  
YATRA.AS  -0.016464  -0.094402   0.000000  
YK6B.F     0.150318  -0.070875  -0.085221  
ZEL.NZ    -0.045614   0.253676   0.055132  
ZEN.BR    -0.051057   0.058447   0.000000  

[1407 rows x 3705 columns]

In [18]:
X_test = X_test.select_dtypes(exclude=["object", "datetime"])
assert len(X_train.columns) == len(X_test.columns)

In [19]:
top20_stocks = pd.Series(model.predict(X_test), X_test.index).sort_values()[-20:].index

/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/opt/homebrew/anaconda3/envs/hackinsey/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtyp